# Trees and graphs

Trees and graphs are important data structures, underpinning a great deal of disparate computing applications, ranging from databases, search, routing, navigation etc. At first blush, might not seem like good candidates for APL's array-based processing. We could, of course, represent such structures in APL like we'd do in any other language, but we'd not play to APL's strengths. Let's look at some options.

In [62]:
⎕IO←0
]box on -s=max

┌→────────────────┐
│Was ON -style=min│
└─────────────────┘

## Trees

A _tree_ is a structure consisting of _nodes_, each holding some information payload, and each of which has exactly 1 parent node, and 0 or more child-nodes. We can stipulate the existence of 0 or 1 _root nodes_, where the parent node is the root node itself.

We know by now that "flat is fast" when it comes to APL. We will strive to keep the tree _structure_ separate from its content. Here's a binary tree (each node has a maximum of two children):

```
      A
     / \
    B   C
   /    |\
  D     E F
       /
      G
```
We can represent this in APL with two vectors, one containing the payloads (the capital letters denoting the nodes in our diagram) and another mapping node index to parent:

In [15]:
nodes←'ABCDEFG'
parents←0 0 0 1 2 2 4

This representation has a couple of advantages. Firstly, the structure is flat, regardless of the number of child nodes a node can have. Secondly, adding nodes anywhere in the tree is an append-only operation. Let's say we wanted to add another node `H` under the `B` node:

In [16]:
nodes←'ABCDEFGH'
parents←0 0 0 1 2 2 4 1  ⍝ Add H node

A common operation on trees (and, indeed, graphs) is to find _paths_ between nodes. Let's say we want to find the sequence of nodes visited when traveling from the root node (`A`) to the node denoted by `G`. We'd go `ACEG`. This becomes a matter of starting at the target node, `G`, and jumping to its parent, `E`, then its parent, `C` and finally its parent is the root `A`. We then reverse the sequence. How can we do this in APL?

Step 1 is to find the index of the target node. That's easy:

In [17]:
target←⎕←nodes⍳'G'

6

Step 2 is to repeatedly visit the parents until the parent node stops changing (we know the root's parent is itself). This sounds easy, too, right? Surely, that's the power operator with a fixed-point, `{something}⍣≡target`? 

Almost. Unfortunately, the fixed-point only returns the _final_ answer:

In [18]:
{⌊⍵÷2}⍣≡32   ⍝ Repeatedly integer-divide 32 by 2 until it no longer changes, 32 16 8 4 2 1 0

0

Getting something that accumulates each intermediate value requires a bit of effort. Fortunately, we can pick the expression directly from [APLCart](https://aplcart.info/?q=scan%20power#):

In [19]:
∆←{⍺←⊢ ⋄ r⊣⍺ ⍺⍺{⍺←⊢ ⋄ r,∘⊂←⍺ ⍺⍺ ⍵}⍣⍵⍵⊃r←⊂⍵}  ⍝ power-scan

We'll hang that off a spare "glyph" so we can pretend this functionality is built-in. Let's see if it works:

In [20]:
idx←⎕←{⍵⊃parents}∆{⍺=0}target

6 4 2 0

Looks right - the first element is the target itself, and the last element is the root. Let's get the full path in terms of the node names:

In [21]:
⊖nodes[idx]

ACEG

We can now combine this into a handy operator,

In [42]:
_path←{⊖⍺⍺∘{⍵⊃⍺}∆{⍺=0}⍵}

In [46]:
p←⎕←(parents _path) nodes⍳'G'
nodes[p]

0 2 4 6

ACEG

Next up, we'll need a way to visit all nodes under a given node. This is called a _traversal_, and can be done in many ways. Let's start with a _breadth-first traversal_. Here's our tree again:

```
      A
     / \
    B   C
   /    |\
  D     E F
       /
      G
```
A breadth-first traversal starting at `C` would yield `C EF G`. A full traversal from the root would give us `A BC DEF G`. 

For this we can utilise our handy _power-scan_ operator, `∆`, again. This time, in each iteration we need to find all children, in turn, or if we flip that around, all nodes which have _me_ as their parent. We could use the _find_ built-in, `⍷`, for that, or an outer product,


TODO wordle←∧/∊

In [90]:
C←nodes⍳'C'
nodes[⍸C⍷parents]
nodes[⍸C∘.=parents]

┌→─┐
│EF│
└──┘

┌→─┐
│EF│
└──┘

We'll use the outer product, as it's convenient if we want to look up several nodes at once.

In [68]:
nodes←'ABCDEFG'          ⍝ Here's our tree again
parents←0 0 0 1 2 2 4

Let's concoct an operator again. A slight complication is that we need to exclude the zero index to avoid an infinite loop if we're starting from the root:

In [87]:
_bft←{¯1↓⍺⍺∘{0~⍨⍸∨⌿⍵∘.=⍺}∆{0=≢⍺},⍵}

In [88]:
p←⎕←(parents _bft)nodes⍳'C'
{nodes[⍵]}¨p
p←⎕←(parents _bft)nodes⍳'A'
{nodes[⍵]}¨p

┌→──────────────┐
│ ┌→┐ ┌→──┐ ┌→┐ │
│ │2│ │4 5│ │6│ │
│ └~┘ └~──┘ └~┘ │
└∊──────────────┘

┌→─────────────┐
│ ┌→┐ ┌→─┐ ┌→┐ │
│ │C│ │EF│ │G│ │
│ └─┘ └──┘ └─┘ │
└∊─────────────┘

┌→──────────────────────┐
│ ┌→┐ ┌→──┐ ┌→────┐ ┌→┐ │
│ │0│ │1 2│ │3 4 5│ │6│ │
│ └~┘ └~──┘ └~────┘ └~┘ │
└∊──────────────────────┘

┌→───────────────────┐
│ ┌→┐ ┌→─┐ ┌→──┐ ┌→┐ │
│ │A│ │BC│ │DEF│ │G│ │
│ └─┘ └──┘ └───┘ └─┘ │
└∊───────────────────┘

What about a _depth_ first traversal? Assuming we process children left to right, the above tree would, from the root, yield a node order of `A B D C E G F`. In the binary tree case, we'd say 

1. Visit node
2. Depth-first traversal of left child, if present
3. Depth-first traversal of right child, if present

which lends itself naturally to a recursive formulation:

In [76]:
_dftr←{⍺←⍬⋄0=≢⍵:⍺⋄(⍺,⊃⍵)∇(0~⍨⍸⍺⍺⍷⍨⊃⍵),1↓⍵}   ⍝ right arg to ∇ is a stack

In [77]:
nodes[(parents _dftr) nodes⍳'A']

┌→──────┐
│ABDCEGF│
└───────┘

Obviously, the difference between depth-first and breadth-first is just the order in which we process the remaining nodes. A recursive formulation of the `_bft` operator above is simply

In [93]:
_bftr←{⍺←⍬⋄0=≢⍵:⍺⋄(⍺,⊃⍵)∇(1↓⍵),0~⍨⍸⍺⍺⍷⍨⊃⍵}  ⍝ right arg to ∇ is a queue

In [94]:
nodes[(parents _bftr)nodes⍳'A']

┌→──────┐
│ABCDEFG│
└───────┘